### Importing necessary Python libiraies 

In [2202]:
# import necessary libraries
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import re
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")  # Ignore all warnings


In [2204]:
#loading dataset into python dataframe
df = pd.read_csv('raw_data.csv')
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


### Going through the dataset

In [2207]:
# going through the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


Order ID, Quantity Ordered, Price and Order date type have to be formatted.

### Cleaning dataset

In [2209]:
# checking the total null values
df.isnull().sum()

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [2211]:
# remove null values from dataset
df.dropna(inplace=True)

In [2213]:
# format data type 
df['Order Date'] = pd.to_datetime(df['Order Date'],errors='coerce')
df['Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'],errors='coerce')
df['Price Each'] = pd.to_numeric(df['Price Each'],errors='coerce')
df['Order ID'] = pd.to_numeric(df['Order ID'],errors='coerce')


In [2214]:
df.isnull().sum()

Order ID            355
Product               0
Quantity Ordered    355
Price Each          355
Order Date          355
Purchase Address      0
dtype: int64

In [2173]:
# droping null values in both Quantity Ordered and Order ID
df = df.dropna(inplace=True)
df['Quantity Ordered'] = df['Quantity Ordered'].astype(int)
df['Order ID'] = df['Order ID'].astype(int)


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# checking duplicate
df.duplicated().value_counts()

In [ ]:
#remove duplicated values from dataset
df.drop_duplicates(inplace=True)

### Data processing 

In [ ]:
# extracting date, hour, day, weekofyear, month, and year from Order_date
df['Hour'] = df['Order Date'].dt.hour # Hour of the day
df['Day'] = df['Order Date'].dt.day_name() # Day of the month
df['Month'] = df['Order Date'].dt.month_name() # name of the mohth in the year eg(january,febuary...)
df['mon'] = df['Order Date'].dt.month # month_position in the year eg(1,2...)
df['Year'] = df['Order Date'].dt.year # year
df['Week_of_Year'] = df['Order Date'].dt.isocalendar().week #week of the year


In [ ]:
#checking number of unique values in Product column
df['Product'].unique()


In [ ]:
# Define category patterns

patterns = {
    'Phone': r'Phone|iPhone',
    'Accessories': r'Cable|Headphones|Batteries',
    'Monitors': r'Monitor',
    'Laptop':r'Laptop',
    'TV':r'TV|Flatscreen',
    'Appliances':r'Dryer|Washing Machine'
    }

# Function to categorize products based on patterns
def categorize_products(products):
    if pd.isnull(products):
        return 'Other'
    for category, pattern in patterns.items():
        if re.search(pattern, products, re.IGNORECASE):
            return category
    return 'Other'

# Apply categorization
df['categorize'] = df['Product'].apply(categorize_products)

In [ ]:
df['categorize'].unique()

In [ ]:
# create revenue column by multiplying Quantity_Ordered  by Price_Each column
df['Revenue'] = df['Quantity Ordered'] * df['Price Each']

In [ ]:
# extracting city from purchase address
df['city'] = df['Purchase Address'].apply(lambda x: x.split(',')[1].strip())
print(df['city'].head())

In [ ]:
# data type well formated, 
df.info()

Now the dataset is well cleaned up properly and processed.

### Explotatrary Data Analysis (EDA)

#### Sales Analysis

##### Best Month for Sales

In [ ]:
# Best Month for Sales
month_sales = df['Month'].value_counts().sort_values(ascending=False).reset_index()
px.bar(month_sales,x='Month',y='count',title='Monthly sales')


 ##### What day of the week has the highest sales?

In [ ]:
# Convert Week_of_Year into Week_of_Month (forcing a max of 4 weeks)
df['Week_of_Month'] = df.groupby(['Year', 'mon'])['Week_of_Year'].rank(method='dense').astype(int)

# If a month has a 5th week, merge it into the 4th week
df.loc[df['Week_of_Month'] > 4, 'Week_of_Month'] = 4  

# Group by Month, Week of Month, and Day of Week to get total Revenue (everyday sales calculation)
weekly_sales = df.groupby(['mon', 'Week_of_Month', 'Day'], as_index=False)['Revenue'].sum()

# Find the Best Sales Day per Week (Day with highest revenue in each week)
best_sales_per_week = weekly_sales.loc[weekly_sales.groupby(['mon', 'Week_of_Month'])['Revenue'].idxmax()]
print(best_sales_per_week.head(10))

# Create a Bar Chart in Plotly
px.bar(best_sales_per_week, x="Week_of_Month", y="Revenue", color="Day", facet_col='mon', title="Best Sales Day in Each Week of Every Month",
      labels={"Week_of_Month": "Week", "Revenue": "Total Sales", "Day": "Best Sales Day"}, text="Day", height=600, width=1400 )

##### Timeline of Day of the Week vs. Revenue

In [ ]:
# Convert Week_of_Year into Week_of_Month (forcing a max of 4 weeks)
df['Week_of_Month'] = df.groupby(['Year', 'mon'])['Week_of_Year'].rank(method='dense').astype(int)

# If a month has a 5th week, merge it into the 4th week
df.loc[df['Week_of_Month'] > 4, 'Week_of_Month'] = 4  

# Group by Month, Week of Month, and Day of Week to get total Revenue (everyday sales calculation)
weekly_sales = df.groupby(['mon', 'Week_of_Month', 'Hour'], as_index=False)['Revenue'].sum()

# Find the Best Sales Day per Week (Day with highest revenue in each week)
best_sales_per_week = weekly_sales.loc[weekly_sales.groupby(['mon', 'Week_of_Month'])['Revenue'].idxmax()]
print(best_sales_per_week.head(15))

# Create a Bar Chart in Plotly
fig = px.bar(best_sales_per_week,x="Week_of_Month",y="Revenue",color="Hour",facet_col='mon',title="Best Sales Hour in Each Week of Every Month",
    labels={"Week_of_Month": "Week ", "Revenue": "Sales", "Hour": "Best SalesHour"},text="Hour",height=600,width=1400)
fig.show()

##### Sales Per Hour

In [ ]:
Sales_per_hour = df.groupby('Hour')['Revenue'].sum().sort_values(ascending=False).reset_index()
print(Sales_per_hour.head())

px.bar(Sales_per_hour,x='Hour',y='Revenue')

#### Product Analysis

##### What Product Sold the Most?

In [ ]:
# what product sold the most
Product_sales = df['Product'].value_counts().sort_values(ascending=False).reset_index()
print(Product_sales.head())
px.bar(Product_sales,x='Product',y='count')

##### Top Products for Each City

#####  Top 5 Product with highest revenue for each city

In [ ]:
# Group by city and product to get total quantity ordered and revenue
Cities_Product = df.groupby(['city', 'Product'], as_index=False).agg({'Quantity Ordered': 'sum','Revenue': 'sum'})

# Rename columns for clarity
Cities_Product.rename(columns={'Quantity Ordered': 'Orders Count'}, inplace=True)

# Function to get top 5 products for each city
def get_top_product(order):
    return order.nlargest(5, 'Orders Count')  # Sort by order count

# Apply function to find top 5 products per city
Cities_Product = Cities_Product.groupby(['city'], group_keys=False).apply(get_top_product)

# Create pivot tables for 'Revenue'
pivot_orders = Cities_Product.pivot_table(index='city', values='Orders Count', columns='Product', fill_value=0).reset_index()
print("Pivot Table: Orders Count")
pivot_orders


In [ ]:

# Convert pivot table to long format
pivot_orders_long = pivot_orders.reset_index().melt(id_vars='city', var_name='Product', value_name='Orders Count')
# Create bar chart
px.bar(pivot_orders_long,x="city",y="Orders Count",color="Product",barmode="group",title="Top 5 Products Ordered in Each City",
    labels={"city": "City", "Orders Count": "Total Orders"},width=1400,height=500)

#####  Top 5 highest Product in each city

In [ ]:
# Create pivot tables for 'Revenue'
pivot_orders = Cities_Product.pivot_table(index='city', values='Revenue', columns='Product', fill_value=0).reset_index()
# Display the pivot tables
print("Pivot Table: Revenue")
pivot_orders


In [ ]:
# convert pivot table to long format
pivot_orders = pivot_orders.melt(id_vars='city',var_name='Product',value_name='Revenue')

px.bar(pivot_orders,x='city',y='Revenue',color='Product',barmode='group',title='Top 5 Products Revenue in Each City',
       labels={'city':'city','Revenue':'Total Revenue'},width=1000,height=500,)    

##### What Products Are Most Often Sold Together?


In [ ]:
from itertools import combinations
from collections import Counter

# Filter orders that contain multiple products
multi_product_orders = df[df['Order ID'].duplicated(keep=False)]
# Group products by 'Order ID'
group_orders = multi_product_orders.groupby('Order ID')['Product'].apply(list)

# Generate product pairs
product_pairs = []
for products in group_orders:
    product_pairs.extend(combinations(sorted(products),2)) # sort to avoid duplicate pair variations
#count most common product pairs
pair_counts = Counter(product_pairs)
#convert to DataFrame for easy analysis
most_common_pairs = pd.DataFrame(pair_counts.most_common(10), columns=["Product Pair", "Count"])

print(most_common_pairs)

##### What Percentage of Orders Include Multiple Products?


In [ ]:
total_orders = df['Order ID'].nunique()  #count total order 
order_count = df['Order ID'].value_counts()      # Count the number of products per order

multiple_id = order_count[order_count>1].count()     #count multiple order
single_order = order_count[order_count == 1].count()    # Get Single Product Orders (where count is 1)

percentage_multiple_order = (multiple_id/total_orders)*100   #find the percentage of multiple orde
percentage_single = 100 -percentage_multiple_order       #percentage of single order

#displays the output 
print(f'''
    Total_orders = {total_orders}\n
    Multiple_order = {multiple_id}\n
    Single_order = {single_order}\n
    Multiple_order % = {percentage_multiple_order}\n
    Single_order % = {percentage_single}
''')


#### Order Value Analysis


##### What Was the Highest Single-Order Value?


In [ ]:
highest_single_values  = df[df['Revenue'] == max(df['Revenue'])]
highest_single_values

##### whict city has the higest Revenue


In [ ]:
city_higest_revenue = df.groupby('city',as_index=False).agg({'Order ID':'count','Revenue':'sum'})
city_higest_revenue.rename(columns={'Order ID':'Order Count'},inplace=True)

px.bar(city_higest_revenue,x='city',y='Revenue',title='Higest city Sales',width=1000,height=500)

In [ ]:
print(city_higest_revenue)
px.bar(city_higest_revenue,x='city',y='Order Count',title='Higest city Sales',width=1000,height=500)

#### State Analysis


###### What Is the Distribution of States?


In [ ]:
city_distribution = df['city'].value_counts().reset_index()
print(city_distribution)
px.pie(city_distribution,values='count',names='city')